# Расчёт среднего времени ответа менеджера

## Задача

Рассчитать среднее время ответа менеджера клиенту в чате

## Дано

Таблица со следующими полями:
- **message_id** - уникальный идентификатор сообщения
- **type** - тип сообщения: outgoing_chat_message – исходящее сообщение от менеджера клиенту, incoming_chat_message – входящее сообщение от клиента менеджеру
- **entity_id** - идентификатор сделки в amoCRM. На каждого клиента открывается отдельная сделка
- **created_by** - идентификатор того, кто написал сообщение. У клиентов всегда 0. 
- **created_at** - время создания сообщения в формате Unix Timestamp

Расчёт должен учитывать следующее: 
- Если в диалоге идут несколько сообщений подряд от клиента или менеджера, то при расчёте времени ответа надо учитывать только первое сообщение из каждого блока; 
- Менеджеры работают с 09:30 до 00:00, поэтому нерабочее время не должно учитываться в расчёте среднего времени ответа, т.е. если клиент написал в 23:59, а менеджер ответил в 09:30 – время ответа равно одной минуте; 
- Ответы на сообщения, пришедшие ночью также нужно учитывать.

## Решение

### 1. Обзор и подготовка данных

In [1]:
#Импорт библиотек
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Создаём датафрейм
data = pd.read_csv('chat_messages.csv')

In [3]:
# Смотрим на результат
data.head(10)

,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199
5,"""01jb478n40pk0c0v1zsk3wtwfr""",incoming_chat_message,37550015,0,1729939920
6,"""01jb3t5vr8rfzv8j2ny519nqzk""",incoming_chat_message,37427859,0,1729926197
7,"""01jb3z6s5gv240x8qg8kj8927v""",incoming_chat_message,37054671,0,1729931470
8,"""01jb3t5rtg6s3eks367rm33rb0""",incoming_chat_message,37546971,0,1729926194
9,"""01jb3t5rtghb56gcbvktr7txx5""",incoming_chat_message,37549489,0,1729926194


In [4]:
# Выводим информацию о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18929 entries, 0 to 18928
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message_id  18929 non-null  object
 1   type        18929 non-null  object
 2   entity_id   18929 non-null  int64 
 3   created_by  18929 non-null  int64 
 4   created_at  18929 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 739.5+ KB


In [5]:
# Переводим тип столбца 'created_at' в datetime и переводим в Московское время
data['created_at'] = pd.to_datetime(data['created_at'], unit='s')
data['created_at'] += pd.Timedelta('03:00:00')

In [6]:
# Оставим в таблице только колонки, необходимые для рассчёта
data.drop(columns = ['message_id', 'type'], inplace=True)

In [7]:
# Смотрим на результат
data.head()

,entity_id,created_by,created_at
0,37556493,0,2024-10-27 19:35:24
1,37549491,0,2024-10-26 10:03:33
2,37531455,0,2024-10-27 11:13:02
3,37553371,0,2024-10-27 17:16:23
4,37548675,0,2024-10-26 10:03:19


Добавим колонки с числом месяца и временем сообщения, чтобы при вычислении разницы времени ответа учитывать рабочее время менеджеров

In [8]:
data['created_day'] = data['created_at'].dt.day
data['created_time'] = data['created_at'].dt.time

In [9]:
# Упорядочим таблицу по entity_id
data.sort_values('entity_id').head(30)

,entity_id,created_by,created_at,created_day,created_time
6639,35810509,10262493,2024-10-26 18:36:46,26,18:36:46
4102,35810509,10262493,2024-10-26 18:21:05,26,18:21:05
6430,35810509,0,2024-10-26 18:34:46,26,18:34:46
3107,35810509,0,2024-10-26 18:19:26,26,18:19:26
17586,35945535,0,2024-10-27 20:38:36,27,20:38:36
17852,35945535,10262493,2024-10-27 20:42:07,27,20:42:07
14989,36099337,0,2024-10-27 16:27:14,27,16:27:14
15039,36099337,11241942,2024-10-27 16:29:37,27,16:29:37
14805,36099337,0,2024-10-27 16:21:56,27,16:21:56
14833,36099337,11241942,2024-10-27 16:25:47,27,16:25:47


Посмотрим, нет ли в таблице `entity_id`, где есть только сообщения от клиента или только сообщения от менеджера

In [10]:
# Посчитаем кол-во уникальных 'created_by' для каждого 'entity_id'
ids = data.groupby('entity_id')['created_by'].nunique()

In [11]:
ids

entity_id
35810509    2
35945535    2
36099337    2
36209333    1
36222379    1
           ..
37557215    2
37557225    1
37557229    1
37557233    1
37557235    1
Name: created_by, Length: 3544, dtype: int64

In [12]:
# выведем уникальные сгруппированные значения
ids.unique()

array([2, 1, 3, 4], dtype=int64)

В таблице присутствуют `entity_id` только с одним полем `created_by`. Для них невозможно подсчитать время ответа. Остаим только те `entity_id`, где 2 или более полей `created_by`.

In [13]:
# Создадим список с entity_id, где кол-во уникальных 'created_by' > 1
ids_list = ids[ids > 1]

In [14]:
ids_list.index

Int64Index([35810509, 35945535, 36099337, 36333761, 36373713, 36406841,
            36472665, 36487759, 36501589, 36586787,
            ...
            37557075, 37557083, 37557097, 37557101, 37557137, 37557147,
            37557149, 37557161, 37557187, 37557215],
           dtype='int64', name='entity_id', length=1581)

Создадим таблицу с entity_id, для которых кол-во уникальных 'created_by' > 1

In [15]:
step_1 = data.loc[data['entity_id'].isin(ids_list.index)]

# Упорядочим таблицу по entity_id и дате создания сообщения
step_1.sort_values(['entity_id', 'created_at'], inplace=True, ignore_index=True)

In [16]:
step_1

,entity_id,created_by,created_at,created_day,created_time
0,35810509,0,2024-10-26 18:19:26,26,18:19:26
1,35810509,10262493,2024-10-26 18:21:05,26,18:21:05
2,35810509,0,2024-10-26 18:34:46,26,18:34:46
3,35810509,10262493,2024-10-26 18:36:46,26,18:36:46
4,35945535,0,2024-10-27 20:38:36,27,20:38:36
...,...,...,...,...,...
16825,37557161,10262493,2024-10-27 23:33:56,27,23:33:56
16826,37557187,7996770,2024-10-27 23:34:49,27,23:34:49
16827,37557187,0,2024-10-27 23:58:20,27,23:58:20
16828,37557215,0,2024-10-27 23:43:28,27,23:43:28


Создадим сводную таблицу по `entity_id` с первым сообщением от клиента и первым сообщением от менеджера

In [17]:
step_2 = step_1.pivot_table(index=['entity_id', 'created_by'],values=['created_at', 'created_day', 'created_time'], aggfunc='first').reset_index()

# Отсортируем таблицу по entity_id и created_at
step_2 = step_2.sort_values(['entity_id', 'created_at'], ignore_index=True)

In [18]:
step_2

,entity_id,created_by,created_at,created_day,created_time
0,35810509,0,2024-10-26 18:19:26,26,18:19:26
1,35810509,10262493,2024-10-26 18:21:05,26,18:21:05
2,35945535,0,2024-10-27 20:38:36,27,20:38:36
3,35945535,10262493,2024-10-27 20:42:07,27,20:42:07
4,36099337,0,2024-10-27 16:21:56,27,16:21:56
...,...,...,...,...,...
3334,37557161,10262493,2024-10-27 23:33:56,27,23:33:56
3335,37557187,7996770,2024-10-27 23:34:49,27,23:34:49
3336,37557187,0,2024-10-27 23:58:20,27,23:58:20
3337,37557215,0,2024-10-27 23:43:28,27,23:43:28


Создадим таблицу с первым сообщением только от клиента

In [19]:
clients = step_2.query('created_by == 0')
clients.reset_index(drop=True, inplace=True)

In [20]:
clients

,entity_id,created_by,created_at,created_day,created_time
0,35810509,0,2024-10-26 18:19:26,26,18:19:26
1,35945535,0,2024-10-27 20:38:36,27,20:38:36
2,36099337,0,2024-10-27 16:21:56,27,16:21:56
3,36333761,0,2024-10-26 19:28:45,26,19:28:45
4,36373713,0,2024-10-27 09:35:47,27,09:35:47
...,...,...,...,...,...
1576,37557147,0,2024-10-27 23:17:43,27,23:17:43
1577,37557149,0,2024-10-27 23:05:09,27,23:05:09
1578,37557161,0,2024-10-27 23:11:16,27,23:11:16
1579,37557187,0,2024-10-27 23:58:20,27,23:58:20


Создадим таблицу с первым сообщением только от менеджера

In [21]:
step_4 = step_2.query('created_by != 0')
step_4.reset_index(drop=True, inplace=True)

In [22]:
step_4

,entity_id,created_by,created_at,created_day,created_time
0,35810509,10262493,2024-10-26 18:21:05,26,18:21:05
1,35945535,10262493,2024-10-27 20:42:07,27,20:42:07
2,36099337,11241942,2024-10-27 16:25:47,27,16:25:47
3,36333761,10262485,2024-10-26 19:33:28,26,19:33:28
4,36373713,7996770,2024-10-27 09:37:38,27,09:37:38
...,...,...,...,...,...
1753,37557147,7996770,2024-10-27 23:16:56,27,23:16:56
1754,37557149,10465254,2024-10-27 23:33:40,27,23:33:40
1755,37557161,10262493,2024-10-27 23:33:56,27,23:33:56
1756,37557187,7996770,2024-10-27 23:34:49,27,23:34:49


Проверим, нет ли в таблице `entity_id` с сообщениями от нескольких менедежров

In [23]:
ids_m = step_4.groupby('entity_id')['created_by'].nunique()

In [24]:
ids_m

entity_id
35810509    1
35945535    1
36099337    1
36333761    1
36373713    1
           ..
37557147    1
37557149    1
37557161    1
37557187    1
37557215    1
Name: created_by, Length: 1581, dtype: int64

In [25]:
# выведем уникальные сгруппированные значения
ids_m.unique()

array([1, 2, 3], dtype=int64)

В таблице с сообщениями только от менеджеров присутствуют `entity_id` с сообщениями от нескольких менеджеров. Для рассчёта времени ответа необходим только первый по времени ответ менеджера.

Создадим сводную таблицу с ответом только от первого менеджера

In [26]:
managers = step_4.pivot_table(index='entity_id', values=['created_by', 'created_at', 'created_day', 'created_time'], aggfunc='first').reset_index()

managers = managers[['entity_id', 'created_by', 'created_at', 'created_day', 'created_time']]

# Отсортируем таблицу по entity_id и created_at
managers = managers.sort_values('entity_id', ignore_index=True)

In [27]:
managers

,entity_id,created_by,created_at,created_day,created_time
0,35810509,10262493,2024-10-26 18:21:05,26,18:21:05
1,35945535,10262493,2024-10-27 20:42:07,27,20:42:07
2,36099337,11241942,2024-10-27 16:25:47,27,16:25:47
3,36333761,10262485,2024-10-26 19:33:28,26,19:33:28
4,36373713,7996770,2024-10-27 09:37:38,27,09:37:38
...,...,...,...,...,...
1576,37557147,7996770,2024-10-27 23:16:56,27,23:16:56
1577,37557149,10465254,2024-10-27 23:33:40,27,23:33:40
1578,37557161,10262493,2024-10-27 23:33:56,27,23:33:56
1579,37557187,7996770,2024-10-27 23:34:49,27,23:34:49


In [28]:
# Проверим результат
managers.groupby('entity_id')['created_by'].nunique().unique()

array([1], dtype=int64)

Объединим таблицы clients и managers по полю `enttiy_id`

In [29]:
# Подготовим таблицы: удалим ненужные столбцы и переименуем столбцы
clients.drop(['created_by', 'created_at'], axis=1, inplace=True)
clients.columns = ['entity_id', 'day_client', 'time_client']
managers.drop('created_at', axis=1, inplace=True)
managers.columns = ['entity_id', 'manager', 'day_manager', 'time_manager']

In [30]:
# Соединяем таблицы
united_data = clients.merge(managers, how='left', on='entity_id')

In [31]:
united_data

,entity_id,day_client,time_client,manager,day_manager,time_manager
0,35810509,26,18:19:26,10262493,26,18:21:05
1,35945535,27,20:38:36,10262493,27,20:42:07
2,36099337,27,16:21:56,11241942,27,16:25:47
3,36333761,26,19:28:45,10262485,26,19:33:28
4,36373713,27,09:35:47,7996770,27,09:37:38
...,...,...,...,...,...,...
1576,37557147,27,23:17:43,7996770,27,23:16:56
1577,37557149,27,23:05:09,10465254,27,23:33:40
1578,37557161,27,23:11:16,10262493,27,23:33:56
1579,37557187,27,23:58:20,7996770,27,23:34:49


### 2. Вычисление времени ответа менеджера

По условия задачи возможны несколько вариантов вычисления времени ответа менеджера:

1. Клиент пишет в рабочее время с 9:30 до 24:00 и менеджер отвечает в этот же день после 9:30
    - Вычисляем разницу по времени
2. Клиент пишет в нерабочее время с 0:00 до 9:30 и менеджер отвечает до 9:30 в этот же день
    - Вычисляем разницу по времени
3. Клиент пишет в нерабочее время с 0:00 до 9:30 и менеджер отвечает после 9:30 в этот же день
    - Время ответа менеджера минус 9:30
4. Менеджер отвечает на следующий день
    - 4.1 если м отв. с 0:00 до 9:30, отнимаем от 23:59:59 время вопроса клиента.
    - 4.2 если м отв. после 9:30, отнимаем от времени ответа менеджера 9:30

In [32]:
# Создадим функцию для вычисления времени ответа менеджера
def response_time(row):
    """
    Вычисление времени ответа менеджера по следующим вариантам:
    
    1. Клиент пишет в рабочее время с 9:30 до 24:00 и менеджер отвечает в этот же день после 9:30
     - Вычисляем разницу по времени
    2. Клиент пишет в нерабочее время с 0:00 до 9:30 и менеджер отвечает до 9:30 в этот же день
     - Вычисляем разницу по времени
    3. Клиент пишет в нерабочее время с 0:00 до 9:30 и менеджер отвечает после 9:30 в этот же день
     - Время ответа менеджера минус 9:30
    4. Если менеджер отвечает на следующий день
     - 4.1 если м отв. с 0:00 до 9:30, отнимаем от 23:59:59 время вопроса клиента.
     - 4.2 если м отв. после 9:30, отнимаем от времени ответа менеджера 9:30
    """
    try:
        # 1. Клиент пишет в рабочее время с 9:30 до 24:00 и менеджер отвечает в этот же день после 9:30
        # Проверка совпадения дней И опроса клиента в рабочее время И ответа менеджера в рабочее время
        if (row['day_client'] == row['day_manager']) and \
        ('09:30:00' < str(row['time_client']) < '23:59:59') and \
        ('09:30:00' < str(row['time_manager']) < '23:59:59'): 
            return datetime.datetime.combine(datetime.date.today(), row['time_manager']) - datetime.datetime.combine(datetime.date.today(), row['time_client'])
        # 2. Клиент пишет в нерабочее время с 0:00 до 9:30 и менеджер отвечает до 9:30 в этот же день
        # Проверка совпадения дней И вопроса клиента в нерабочее время И ответа менеджера в нерабочее время
        elif (row['day_client'] == row['day_manager']) and \
        ('00:00:00' < str(row['time_client']) < '09:30:00') and \
        ('00:00:00' < str(row['time_manager']) < '09:30:00'): 
            return datetime.datetime.combine(datetime.date.today(), row['time_manager']) - datetime.datetime.combine(datetime.date.today(), row['time_client'])
        # 3. Клиент пишет в нерабочее время с 0:00 до 9:30 и менеджер отвечает после 9:30 в этот же день
        # Проверка совпадения дней И вопроса клиента в нерабочее время И ответа менеджера в рабочее время
        elif (row['day_client'] == row['day_manager']) and \
        ('00:00:00' < str(row['time_client']) < '09:30:00') and \
        ('09:30:00' < str(row['time_manager']) < '23:59:59'): 
            return datetime.datetime.combine(datetime.date.today(), row['time_manager']) - datetime.datetime.combine(datetime.date.today(), datetime.datetime.strptime('09:30:00','%H:%M:%S').time())
        # 4.1 Менеджер отвечает на следующий день в нерабочее время
        # Проверка несовпадения дней И ответа менеджера в нерабочее время с 0:00 до 9:30
        elif (row['day_client'] != row['day_manager']) and ('00:00:00' < str(row['time_manager']) < '09:30:00'):
            return datetime.datetime.combine(datetime.date.today(), datetime.datetime.strptime('23:59:59','%H:%M:%S').time()) - datetime.datetime.combine(datetime.date.today(), row['time_client'])
        # 4.2 Менеджер отвечает на следующий день в рабочее время
        # Проверка несовпадения дней И если м отв. после 9:30, отнимаем от времени ответа менеджера 9:30
        elif (row['day_client'] != row['day_manager']) and ('09:30:00' < str(row['time_manager'])):
            return datetime.datetime.combine(datetime.date.today(), row['time_manager']) - datetime.datetime.combine(datetime.date.today(), datetime.datetime.strptime('09:30:00','%H:%M:%S').time())
    except:
        pass

In [33]:
# Применим функцию
united_data['response_time'] = united_data.apply(response_time, axis=1)

In [34]:
# Смотрим на результат
united_data

,entity_id,day_client,time_client,manager,day_manager,time_manager,response_time
0,35810509,26,18:19:26,10262493,26,18:21:05,0 days 00:01:39
1,35945535,27,20:38:36,10262493,27,20:42:07,0 days 00:03:31
2,36099337,27,16:21:56,11241942,27,16:25:47,0 days 00:03:51
3,36333761,26,19:28:45,10262485,26,19:33:28,0 days 00:04:43
4,36373713,27,09:35:47,7996770,27,09:37:38,0 days 00:01:51
...,...,...,...,...,...,...,...
1576,37557147,27,23:17:43,7996770,27,23:16:56,-1 days +23:59:13
1577,37557149,27,23:05:09,10465254,27,23:33:40,0 days 00:28:31
1578,37557161,27,23:11:16,10262493,27,23:33:56,0 days 00:22:40
1579,37557187,27,23:58:20,7996770,27,23:34:49,-1 days +23:36:29


В таблице присутствуют отрицательные результаты вычисления времени ответа. В данных `entity_id` ответ менеджера был раньше вопроса клиента. Уберём entity_id с отрицательными значениями 'response_time'

In [35]:
# Если response_time начинается со знака "-", добавим индекс строки в список для удаления
remove_list = []
for i in range(len(united_data)):
    if str(united_data['response_time'][i])[0] == '-':
        remove_list.append(i)

In [36]:
# Удалим из таблицы индексы из полученного списка
united_data.drop(remove_list, axis=0, inplace=True)

In [37]:
united_data

,entity_id,day_client,time_client,manager,day_manager,time_manager,response_time
0,35810509,26,18:19:26,10262493,26,18:21:05,0 days 00:01:39
1,35945535,27,20:38:36,10262493,27,20:42:07,0 days 00:03:31
2,36099337,27,16:21:56,11241942,27,16:25:47,0 days 00:03:51
3,36333761,26,19:28:45,10262485,26,19:33:28,0 days 00:04:43
4,36373713,27,09:35:47,7996770,27,09:37:38,0 days 00:01:51
...,...,...,...,...,...,...,...
1574,37557101,27,22:42:30,11448610,27,22:54:03,0 days 00:11:33
1575,37557137,27,23:02:07,10262513,27,23:12:03,0 days 00:09:56
1577,37557149,27,23:05:09,10465254,27,23:33:40,0 days 00:28:31
1578,37557161,27,23:11:16,10262493,27,23:33:56,0 days 00:22:40


Считаем среднее время ответа response_time для каждого менеджера

In [38]:
# Создаём сводную таблицу
result = united_data.pivot_table(index = 'manager', values = 'response_time', aggfunc='mean').reset_index()

# Переименуем столбцы
result.columns = ['manager', 'avg_response_time']

# Сортируем таблцу в порядке возрастания времени ответа
result = result.sort_values('avg_response_time', ascending=True, ignore_index=True)

# Окргуляем response_time до секунд
result['avg_response_time'] = pd.Timestamp('today').normalize() + result['avg_response_time']
result['avg_response_time'] = result['avg_response_time'].dt.round('s')
result['avg_response_time'] = result['avg_response_time'].dt.time

In [39]:
# Результат
result

,manager,avg_response_time
0,6780177,00:05:50
1,7417617,00:06:29
2,8558094,00:06:58
3,11241942,00:09:00
4,10465254,00:09:20
5,10262497,00:09:57
6,7408305,00:11:12
7,11490278,00:11:24
8,11394126,00:12:23
9,11550890,00:13:02
